# P2P Blockchain-Based Energy Market - Demo

This notebook demonstrates the key features of the blockchain-based energy trading system.

## Features Demonstrated:
1. Account Creation
2. Balance Management (ETH & Power)
3. Transaction Processing
4. Block Mining
5. Blockchain Validation


In [ ]:
# Import required modules
import sys
sys.path.append('.')

from Blockchain import Blockchain, log_change
import account_manager
import sqlite3
import json
from datetime import datetime


## 1. Initialize the Blockchain


In [ ]:
# Create a new blockchain instance
# Note: Set reset_chain=True to start from scratch
blockchain = Blockchain(reset_chain=True)

print("Blockchain initialized!")
print(f"Genesis block created: Block #{blockchain.last_block['index']}")
print(f"Current chain length: {len(blockchain.chain)}")


## 2. Create Accounts


In [ ]:
# Create sample accounts
alice = account_manager.create_account("alice")
bob = account_manager.create_account("bob")
charlie = account_manager.create_account("charlie")

print("Accounts created:")
print(f"\nAlice - ID: {alice['id'][:8]}..., Balance: {alice['balance']} ETH, Power: {alice['power_balance']} kWh")
print(f"Bob - ID: {bob['id'][:8]}..., Balance: {bob['balance']} ETH, Power: {bob['power_balance']} kWh")
print(f"Charlie - ID: {charlie['id'][:8]}..., Balance: {charlie['balance']} ETH, Power: {charlie['power_balance']} kWh")


## 3. Fund Accounts with ETH


In [ ]:
# Add ETH to accounts
account_manager.update_balance("alice", 10.0)
account_manager.update_balance("bob", 15.0)
account_manager.update_balance("charlie", 20.0)

# Verify balances
alice_account = account_manager.get_account("alice")
bob_account = account_manager.get_account("bob")
charlie_account = account_manager.get_account("charlie")

print("ETH Balances:")
print(f"Alice: {alice_account['balance']} ETH")
print(f"Bob: {bob_account['balance']} ETH")
print(f"Charlie: {charlie_account['balance']} ETH")


## 4. Generate Power (Energy) for Accounts


In [ ]:
# Add power to accounts (simulating energy generation)
account_manager.update_power_balance("alice", 200.0)  # Alice has solar panels
account_manager.update_power_balance("bob", 150.0)   # Bob has wind turbine
account_manager.update_power_balance("charlie", 100.0)  # Charlie is a consumer

# Verify power balances
alice_account = account_manager.get_account("alice")
bob_account = account_manager.get_account("bob")
charlie_account = account_manager.get_account("charlie")

print("Power Balances:")
print(f"Alice: {alice_account['power_balance']} kWh (Producer)")
print(f"Bob: {bob_account['power_balance']} kWh (Producer)")
print(f"Charlie: {charlie_account['power_balance']} kWh (Consumer)")


## 5. Execute Energy Trading Transactions


In [ ]:
# Scenario: Alice sells 50 kWh to Charlie at 0.001 ETH/kWh
power_amount = 50.0
price_per_kwh = 0.001
total_cost = power_amount * price_per_kwh

print(f"\nTransaction: Alice → Charlie")
print(f"Power: {power_amount} kWh")
print(f"Price: {price_per_kwh} ETH/kWh")
print(f"Total Cost: {total_cost} ETH")

# Check if Charlie has enough ETH
charlie_account = account_manager.get_account("charlie")
if charlie_account['balance'] >= total_cost:
    print(f"\nCharlie has sufficient balance: {charlie_account['balance']} ETH")
else:
    print(f"\nCharlie has insufficient balance: {charlie_account['balance']} ETH")
    print("Transaction cancelled")


In [ ]:
# Create transaction in blockchain
seller = "alice"
buyer = "charlie"

# Update balances
account_manager.update_balance(buyer, -total_cost)  # Buyer pays
account_manager.update_balance(seller, total_cost)  # Seller receives

account_manager.update_power_balance(seller, -power_amount)  # Seller loses power
account_manager.update_power_balance(buyer, power_amount)  # Buyer gains power

# Add transaction to blockchain
blockchain.new_transaction_seller(seller, buyer, power_amount, price_per_kwh)

# Get updated balances
alice_final = account_manager.get_account("alice")
charlie_final = account_manager.get_account("charlie")

print("\nTransaction Completed!")
print(f"\nAlice - New Balance: {alice_final['balance']} ETH, Power: {alice_final['power_balance']} kWh")
print(f"Charlie - New Balance: {charlie_final['balance']} ETH, Power: {charlie_final['power_balance']} kWh")


## 6. Mine a Block


In [ ]:
# Mine a new block containing the pending transactions
last_block = blockchain.last_block
last_proof = last_block['proof']

print(f"Mining block...")
print(f"Last proof: {last_proof}")

# Find the new proof (Proof of Work)
proof = blockchain.proof_of_work(last_proof)
print(f"New proof found: {proof}")

# Create the new block
previous_hash = blockchain.hash(last_block)
new_block = blockchain.new_block(proof, previous_hash)

print(f"\nBlock #{new_block['index']} mined successfully!")
print(f"Block Hash: {new_block['block_hash']}")
print(f"Previous Hash: {new_block['previous_hash']}")
print(f"Transactions: {len(new_block['transactions'])}")


## 7. View the Blockchain


In [ ]:
# Display the full blockchain
print("=== FULL BLOCKCHAIN ===\n")
for i, block in enumerate(blockchain.chain):
    print(f"Block #{block['index']}")
    print(f"  Timestamp: {block['timestamp']}")
    print(f"  Proof: {block['proof']}")
    print(f"  Hash: {block['block_hash'][:20]}...")
    print(f"  Previous Hash: {block['previous_hash'][:20]}...")
    print(f"  Transactions: {len(block['transactions'])}")
    
    for j, tx in enumerate(block['transactions']):
        print(f"    Transaction {j+1}:")
        print(f"      Seller: {tx['Seller']}")
        print(f"      Buyer: {tx['Buyer']}")
        print(f"      Power: {tx['Power']} kWh")
        print(f"      Price: {tx['Price']} ETH/kWh")
    print("\n")


## 8. Validate Blockchain Integrity


In [ ]:
# Validate the blockchain
is_valid = blockchain.validate_chain()

if is_valid:
    print("✓ Blockchain is VALID")
    print("  - All block hashes are consistent")
    print("  - Proof-of-work is valid for all blocks")
else:
    print("✗ Blockchain is INVALID")
    print("  - Hash mismatch detected!")


## 9. View All Accounts and Balances


In [ ]:
# Get all accounts and their balances
all_accounts = account_manager.get_all_accounts()

print("=== ALL ACCOUNTS ===\n")
for account in all_accounts:
    print(f"Name: {account['name']}")
    print(f"  ID: {account['id'][:8]}...")
    print(f"  ETH Balance: {account['balance']} ETH")
    print(f"  Power Balance: {account['power_balance']} kWh")
    print(f"  Created: {account['created_at']}")
    print("\n")


## 10. Advanced: Multiple Transactions


In [ ]:
# Add more transactions to the pending pool
print("Adding multiple transactions...")

# Transaction 2: Bob sells to Charlie
transaction2_power = 30.0
transaction2_price = 0.0015
transaction2_total = transaction2_power * transaction2_price

# Update balances
account_manager.update_balance("charlie", -transaction2_total)
account_manager.update_balance("bob", transaction2_total)
account_manager.update_power_balance("bob", -transaction2_power)
account_manager.update_power_balance("charlie", transaction2_power)

# Add to blockchain
blockchain.new_transaction_seller("bob", "charlie", transaction2_power, transaction2_price)

print(f"Transaction 2: Bob sold {transaction2_power} kWh to Charlie for {transaction2_total} ETH")

# Transaction 3: Alice sells to Bob
transaction3_power = 40.0
transaction3_price = 0.002
transaction3_total = transaction3_power * transaction3_price

# Update balances
account_manager.update_balance("bob", -transaction3_total)
account_manager.update_balance("alice", transaction3_total)
account_manager.update_power_balance("alice", -transaction3_power)
account_manager.update_power_balance("bob", transaction3_power)

# Add to blockchain
blockchain.new_transaction_seller("alice", "bob", transaction3_power, transaction3_price)

print(f"Transaction 3: Alice sold {transaction3_power} kWh to Bob for {transaction3_total} ETH")
print(f"\nPending transactions: {len(blockchain.current_transactions)}")


## 11. Mine Another Block


In [ ]:
# Mine Another Block
last_block = blockchain.last_block
last_proof = last_block['proof']

proof = blockchain.proof_of_work(last_proof)
previous_hash = blockchain.hash(last_block)
new_block = blockchain.new_block(proof, previous_hash)

print(f"Block #{new_block['index']} mined with {len(new_block['transactions'])} transactions")
print(f"Block Hash: {new_block['block_hash'][:30]}...")

# Validate again
is_valid = blockchain.validate_chain()
print(f"\nBlockchain valid: {is_valid}")
print(f"Total blocks: {len(blockchain.chain)}")


## 12. Final Summary


In [ ]:
# Get final account states
all_accounts = account_manager.get_all_accounts()

print("=== FINAL ACCOUNT STATES ===\n")
total_eth = 0
total_power = 0

for account in all_accounts:
    print(f"{account['name'].upper()}")
    print(f"  ETH: {account['balance']}")
    print(f"  Power: {account['power_balance']} kWh")
    total_eth += account['balance']
    total_power += account['power_balance']

print(f"\nTotal ETH in system: {total_eth}")
print(f"Total Power in system: {total_power} kWh")
print(f"\nBlockchain Status: {len(blockchain.chain)} blocks, Chain Valid: {blockchain.validate_chain()}")


---

## Conclusion

This demo has shown:
1. ✓ Account creation with cryptographic keys
2. ✓ Balance management (ETH and Power)
3. ✓ Energy trading transactions
4. ✓ Block mining with proof-of-work
5. ✓ Blockchain integrity validation
6. ✓ Transaction history and audit trail

The system successfully demonstrates a functional peer-to-peer energy market using blockchain technology!
